In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Check if GPU is available if not use cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
all_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.ImageFolder(root='./Car_Brand_Logos/Train', transform=all_transforms)
test_set = torchvision.datasets.ImageFolder(root='./Car_Brand_Logos/Test', transform=all_transforms)

train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = DataLoader(test_set, batch_size=8,shuffle=False)

classes = ('hyundai', 'lexus', 'mazda', 'mercedes', 'opel', 'skoda', 'toyota', 'volkswagen')

In [4]:
print('Number of images in the training dataset:', len(train_set))
print('Number of images in the testing dataset:', len(test_set))

Number of images in the training dataset: 2512
Number of images in the testing dataset: 400


In [5]:
class ConvNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Pooling layer with kernel size of 2
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))

    def forward(self, out):
        out = F.relu(self.conv1(out))  # Apply first conv layer
        out = self.pool(out)            # Apply first pooling
        out = F.relu(self.conv2(out))
        out = self.pool(out)

        out = torch.flatten(out, 1)
        out = self.dropout(out)

        out = F.relu(self.fc1(out))
        out = self.dropout(out)
        out = F.relu(self.fc2(out))
        out = F.log_softmax(self.fc3(out), dim=1)
        return out

# Initialize the network and move it to the device
model = ConvNeuralNet().to(device)

In [ ]:
# Here it is the loss function and optimizer
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
for epoch in range(epochs):
    model.train()
    runloss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()

        runloss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[{epoch + 1}/{epochs}, {i + 1:5d}] loss: {runloss / 100:.3f}')
            runloss = 0.0

print('Finished Training')

torch.save(model.state_dict(), 'model5.pth')
print('Model saved as model5.pth')

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct // total} %')